<a href="https://colab.research.google.com/github/37stu37/rnc2/blob/master/temporal_network_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%%capture
!pip install memory_profiler

In [0]:
import numpy as np
import pandas as pd
from pathlib import Path
import memory_profiler as mem_profile
import time
import sys

pd.options.mode.chained_assignment = None  # default='warn'

In [0]:
# import data
folder = Path('/content/drive/My Drive/04_Cloud/01_Work/GNS/008_FFE/runs')
edge_file = folder / 'data' / 'Copy of edge_data.parquet'
wind_file = folder / 'data' / 'Copy of GD_wind.csv'

wind_data = pd.read_csv(wind_file) 
edgelist = pd.read_parquet(edge_file, engine='pyarrow')

In [0]:
# %%timeit
def wind_scenario(wind_data):
      i = np.random.randint(0, wind_data.values.shape[0])
      w = wind_data.values[i, 2]
      dist = wind_data.values[i, 1]
      b = wind_data.values[i, 3]
      bear_max = b + 45  # wind direction
      bear_min = b - 45
      if b == 360:
          bear_max = 45
      if b <= 0:  # should not be necessary
          bear_min = 0
      if b == 999:
          bear_max = 999
          bear_min = 0
      return bear_max, bear_min, dist # wind characteristics, bearing and distance


def ignition(edges=edgelist):
    rng = np.random.uniform(0, 1, size=edges.values.shape[0])
    mask = rng < edges.IgnProb_bl.values
    NewActiveEdges = edges[mask]
    return NewActiveEdges


def mask(t, activeEdges_d, listActivatedSources_d, w_b_max, w_b_min, w_d):
    if t==0: # special case at time=0
        return activeEdges_d
    else:
        mask = (activeEdges_d.bearing.values < w_b_max) & (activeEdges_d.bearing.values < w_b_min) & (activeEdges_d.distance < w_d)
        NewActiveEdges = activeEdges_d[mask]
        NewActiveEdges = NewActiveEdges[~NewActiveEdges.source.isin(listActivatedSources_d)]
        return NewActiveEdges


def propagation(activeEdges_d, edges=edgelist):
    NewActiveEdges = edges[edges.source.isin(activeEdges_d.target)]
    return NewActiveEdges


**Main**


---



In [12]:
# %%time
listScenarioDataframes = []
n=10
for scenario in range(n):
    # initial setup
    condition = True
    # listScenarioDataframes = []
    listActivatedSources = []
    time = 0 
    # wind conditions
    w_bearing_max, w_bearing_min, w_distance = wind_scenario(wind_data)
    # ignition / initial state and edges selection
    ActiveEdges = ignition()
    if ActiveEdges.empty:
        continue
    while condition:
        ActiveEdges = mask(time, ActiveEdges, listActivatedSources, w_bearing_max, w_bearing_min, w_distance)
        if ActiveEdges.empty:
            break
        listScenarioDataframes.append(ActiveEdges)
        listActivatedSources.extend(ActiveEdges.source.values)
        ActiveEdges = propagation(ActiveEdges)
        # advance time
        time += 1
    print("scenario : {} time : {} memory : {} MB".format(scenario, time, mem_profile.memory_usage()))
    print("size of dataframe list = {}".format(sys.getsizeof(listScenarioDataframes)))
    if sys.getsizeof(listScenarioDataframes) > 5000: # flush out if memory more than 5GB
        Activations = pd.concat(listScenarioDataframes)
        Activations.to_parquet(folder / 'output' / 'scenario{}_time{}_Activations.parquet'.format(scenario, time), engine='pyarrow')
        print("flushing out memory")
        listScenarioDataframes = []
Activations = pd.concat(listScenarioDataframes)
Activations.to_parquet(folder / 'output' / 'scenario{}_time{}_Activations.parquet'.format(scenario, time), engine='pyarrow') 

scenario : 0 time : 21 memory : [560.15234375] MB
size of dataframe list = 264
scenario : 1 time : 36 memory : [560.15234375] MB
size of dataframe list = 528
scenario : 2 time : 1 memory : [560.15234375] MB
size of dataframe list = 528
scenario : 3 time : 52 memory : [560.8984375] MB
size of dataframe list = 1072
scenario : 4 time : 41 memory : [579.578125] MB
size of dataframe list = 1448
scenario : 5 time : 17 memory : [564.06640625] MB
size of dataframe list = 1448
scenario : 6 time : 50 memory : [564.06640625] MB
size of dataframe list = 1928
scenario : 7 time : 20 memory : [564.06640625] MB
size of dataframe list = 2216
scenario : 8 time : 16 memory : [564.06640625] MB
size of dataframe list = 2216
scenario : 9 time : 16 memory : [564.06640625] MB
size of dataframe list = 2536
